### Import Libraries

Import necessary libraries

In [27]:
import glob
import os
import librosa
import librosa.display
import time
import seaborn as sns
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import skimage.io
import re
from IPython.display import display, Image
%matplotlib inline
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import tensorflow as tf

In [28]:
input_dir = '../input/audio-features/mel_trim_4x4'

In [29]:
images = []
labels = []

for dirname, _, files in os.walk(input_dir):
    for file in files:
        file_path = os.path.join(dirname, file)
        images.append(file_path)
        labels.append(file_path.split('-')[1].split('/')[-1])
        

In [30]:
di = {
    'image': images,
    'label': labels
}
df = pd.DataFrame(di)

In [31]:
df.shape

In [32]:
df.head()

In [33]:
df.columns

In [34]:
np.unique(df['label'])

In [35]:
import random

index = random.randint(0, 2800)

plt.imshow(plt.imread(df['image'][index]))
plt.title('emotion: ' + df['label'][index])

## Shuffling data#reindexing to shuffle the data at random

In [36]:
#reindexing to shuffle the data at random
df = df.reindex(np.random.permutation(df.index))
df.reset_index(drop=True, inplace=True)

## Splitting Dataset

In [37]:
# Defines ratios, w.r.t. whole dataset.
ratio_train = 0.7
ratio_val = 0.15
ratio_test = 0.15

# Produces test split.
remaining, test = train_test_split(
    df, test_size=ratio_test, stratify=df['label'])

# Adjusts val ratio, w.r.t. remaining dataset.
ratio_remaining = 1 - ratio_test
ratio_val_adjusted = ratio_val / ratio_remaining

# Produces train and val splits.
train, val = train_test_split(
    remaining, test_size=ratio_val_adjusted, stratify=remaining['label'])

In [38]:
len(train), len(val), len(test)

In [39]:
train['label'].value_counts()

In [40]:
val['label'].value_counts()

In [41]:
test['label'].value_counts()

In [42]:
import PIL

image = PIL.Image.open(df['image'][2])

width, height = image.size

print(width, height)
display(image)

In [43]:
from keras.preprocessing.image import ImageDataGenerator

In [44]:
datagen = ImageDataGenerator() # Preprocessing included with the model
# datagen = ImageDataGenerator(rescale=1.0/255.0)
# datagen = ImageDataGenerator(samplewise_center=True)
# datagen = ImageDataGenerator(samplewise_center=True, samplewise_std_normalization=True)

In [45]:
train_set = datagen.flow_from_dataframe(
    dataframe=train,
#     directory="face_mask_detection/train",
    x_col="image",
    y_col="label",
    batch_size=32,
#     shuffle=True,
    class_mode="categorical",
    target_size=(width, height))

In [46]:
val_set = datagen.flow_from_dataframe(
    dataframe=val,
#     directory="face_mask_detection/train",
    x_col="image",
    y_col="label",
    batch_size=32,
#     shuffle=True,
    class_mode="categorical",
    target_size=(width, height))

In [47]:
test_set = datagen.flow_from_dataframe(
    dataframe=test,
#     directory="face_mask_detection/train",
    x_col="image",
    y_col="label",
    batch_size=32,
    shuffle=False,
    class_mode="categorical",
    target_size=(width, height))

In [48]:
def evaluationReport(hist, model):
    plt.plot(hist.history['loss'], label='train loss')
    plt.plot(hist.history['val_loss'], label='val loss')
    plt.legend()
    plt.show()
    
    plt.plot(hist.history['accuracy'], label='train accuracy')
    plt.plot(hist.history['val_accuracy'], label='val_accuracy')
    plt.legend()
    plt.show()
    
    labels = np.unique(df['label'])
    
    x, y = zip(*(test_set[i] for i in range(len(test_set))))
    x_test, y_test = np.vstack(x), np.vstack(y)
    
    print('Evaluation on the test set:')
    result = model.evaluate(x_test, y_test, batch_size=32)
    print("Test Loss, Test Acc:", result)
    
    preds = model.predict(test_set)
    
    y_pred = np.argmax(preds, axis=1)
    print()
    print(classification_report(test_set.classes,y_pred))

    print('Confusion Matrix')
    # Confusion Matrix
    cm = confusion_matrix(test_set.classes, y_pred)
    # print(cm)
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, fmt='g', ax=ax);
    # labels, title and ticks
    ax.set_xlabel('Predicted labels');
    ax.set_ylabel('True labels'); 
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels(labels);
    ax.yaxis.set_ticklabels(labels);
    

In [49]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

STEP_SIZE_TRAIN = train_set.n//train_set.batch_size
STEP_SIZE_VALID = val_set.n//val_set.batch_size
STEP_SIZE_TEST = test_set.n//test_set.batch_size

base_lr = 1e-4

es_callback = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1)

mc = ModelCheckpoint('./mode_ckpt', save_best_only=True, save_weights_only=True, verbose=1)

lrs = ReduceLROnPlateau(monitor="val_loss",
    factor=0.5,
    patience=3,
    cooldown=0,
    verbose=1,
    mode="auto",
    min_lr=1e-12)

# MobileNet Large

In [50]:
from tensorflow.keras.applications import MobileNetV3Large

In [51]:
image_size = (width, height, 3)

mobile_net_v3_large = MobileNetV3Large(
    input_shape=image_size,
    alpha=1.0,
    minimalistic=False,
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    classes=6,
    pooling=None,
    dropout_rate=0.2,
    classifier_activation='softmax',
    include_preprocessing=True
)


for layer in mobile_net_v3_large.layers:
    layer.trainable = False

mnlv3 = Sequential([
  mobile_net_v3_large,
#   tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
#   tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(units=6, activation='softmax')
])

mnlv3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_lr), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
mnlv3.summary()

In [52]:
mnlv3_hist = mnlv3.fit(
  train_set,
  validation_data=val_set,
  epochs=500,
  callbacks=[es_callback, lrs],
  steps_per_epoch=STEP_SIZE_TRAIN,
  validation_steps=STEP_SIZE_VALID,
  use_multiprocessing=True,
  workers=10
)

In [53]:
# mnlv3.save('./mnlv3')

In [54]:
evaluationReport(mnlv3_hist, mnlv3)

## Fine-tuning

In [55]:
mobile_net_v3_large.trainable = True

for layer in mobile_net_v3_large.layers[:210]:
    layer.trainable = False

In [56]:
mnlv3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_lr/100), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
mnlv3.summary()

In [ ]:
mnlv3_hist_ft = mnlv3.fit(
  train_set,
  validation_data=val_set,
  epochs=500,
  callbacks=[es_callback, lrs],
  steps_per_epoch=STEP_SIZE_TRAIN,
  validation_steps=STEP_SIZE_VALID,
  use_multiprocessing=True,
  workers=10
)

In [ ]:
evaluationReport(mnlv3_hist_ft, mnlv3)